In [139]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [217]:
kit={}
source=requests.get("https://www.exalpha.com/product/brdu-chemiluminescent-cell-proliferation-assay-kit-2/").text
soup=BeautifulSoup(source,'lxml')
kit.update({"Kit Manual":})

In [232]:
urls=[]
for i in range(1):
    #https://www.exalpha.com/products/page/2/?filter_maincategory=secondary-antibodies
    #urlz="https://www.exalpha.com/products/page/"+str(i+1)+"/?filter_maincategory=secondary-antibodies"
    urlz="https://www.exalpha.com/products/?filter_maincategory=kits"
    source=requests.get(urlz).text
    soup=BeautifulSoup(source,'lxml')
    s=soup.find('table')
    s=s.findAll('a')
    for url in s:
        urls.append(url['href'])
urls

['https://www.exalpha.com/product/brdu-cell-proliferation-assay-kit/',
 'https://www.exalpha.com/product/brdu-cell-proliferation-assay-kit-2/',
 'https://www.exalpha.com/product/brdu-cell-proliferation-assay-kit-3/',
 'https://www.exalpha.com/product/brdu-chemiluminescent-cell-proliferation-assay-kit/',
 'https://www.exalpha.com/product/brdu-chemiluminescent-cell-proliferation-assay-kit-2/',
 'https://www.exalpha.com/product/brdu-chemiluminescent-cell-proliferation-assay-kit-3/',
 'https://www.exalpha.com/product/brdu-immunohistochemistry-kit/',
 'https://www.exalpha.com/product/dna-fragmentation-detection-kit/',
 'https://www.exalpha.com/product/dna-fragmentation-detection-kit-2/',
 'https://www.exalpha.com/product/eggspress-igy-purification-kit-20-yolks/',
 'https://www.exalpha.com/product/eggspress-igy-purification-kit-40-yolks/',
 'https://www.exalpha.com/product/eggspress-igy-purification-kit-5-yolks/',
 'https://www.exalpha.com/product/eggspress-igy-purification-kit-80-yolks/',
 

In [233]:
len(urls)

14

In [251]:
def kits(soup):
    kit={}
    try:

        kit_manual=soup.find('a',text='Click here to download the Kit Manual')['href']
        kit_manual="https://www.exalpha.com"+kit_manual
        kit.update({"Kit Manual_link":kit_manual})
        #kit
    except:
        pass
    return kit
def pro_para(soup):
    a=[]
    para_data={}
    safety={}
    para=soup.findAll('p')

    for p in para:
        try:
            strong=p.find("strong").text
            paras_text=p.text
            para_data.update({strong:paras_text})
        except:
            pass
        try:
            alpha=p.findAll('a')
            for double_a  in alpha:
                
                a.append(double_a['href'])
        except:
            pass

        try:
            ii=p.find('i')
            iip=p.text
            para_data.update({ii:iip})

        except:
            pass
    for d,x in enumerate(a):
        if '.pdf' in x:
            safety.update({"Safety Datasheet_"+str(d+1):x})
    return para_data,safety 

def product_sku(soup):
    sku_number={}
    try:
        sk=soup.find('span',{"class":"sku"}).text
        sku_number.update({"Sku":sk})
    except:
        pass
    return sku_number

def Category_pro(soup):
    
    Category_type={}
    try:
        cat=soup.find('span',{"class":"posted_in"}).text
        Category_type.update({"Categories":cat})
        #Category_type
    except:
        pass
    return Category_type

def table(soup):
    t_data={}
    try:
        table=soup.find('table')
        table=table.findAll('tr')
        for tr in table:
            tabtd=tr.findAll('td')
            try:
                t_data.update({tabtd[0].text:tabtd[1].text})
            except:
                pass
    except:
        pass
    return t_data  

def product_prices(soup):
    prices=[]
    try:
        script=soup.findAll("script",{"type":"application/ld+json"})
        #print(script)
        for s in script:
            s=str(s)
            s=s.split(",")
            for i in s:
                tamp={}
                if '"price":' in i:
                    prize=i
                    prize=prize.split(':')
                    prize=prize[-1]
                    tamp.update({"Price":prize})
                    prices.append(tamp)
    except:
        pass
    return prices 

def pdf(soup):
    datasheet={}
    try:

        image_pdf=soup.find('a',{"class":"noslimstat"})
        image_pdf=image_pdf.find('img')["src"]
        datasheet.update({"Datasheet":image_pdf})
    except:
        pass
    return datasheet

def title(soup):
    pro_tit={}
    try:
        
        titiless=soup.find('h1',{"class":"product_title entry-title"}).text
        pro_tit.update({"Product Title":titiless})
    except:
        pass
    return pro_tit
        
def product_images(soup):
    
    images_pro={}
    try:
        
        img_table=soup.findAll('table')
        for img in img_table:
            images=img.findAll('img')
            if len(images)>0:
                for n,i in enumerate(images):
                    images_pro.update({"Image_"+str(n+1):i['src'],"Image_"+str(n+1)+"_description":i["title"]})
    except:
        pass
    return images_pro    

In [29]:
url="https://www.exalpha.com/product/swine-igg/"
source=requests.get(urls_buffer[1]).text
soup=BeautifulSoup(source,'lxml')

In [252]:
empty=[]
for num,x in enumerate(urls):
    source=requests.get(x).text
    soup=BeautifulSoup(source,'lxml')
    sku=product_sku(soup)
    p_para,safe=pro_para(soup)
    prices=product_prices(soup)
    imgg=product_images(soup)
    cat=Category_pro(soup)
    tab=table(soup)
    tit=title(soup)
    dsheet=pdf(soup)
    kitty=kits(soup)
    for price in prices:
        if len(tit)>0:
            price.update(tit)
        if len(sku)>0:
            price.update(sku)
        price.update({"Product Url":x})    
        if len(cat)>0:
            price.update(cat)
        if len(tab)>0:
            price.update(tab)
        if len(p_para)>0:
            price.update(p_para)
        if len(safe)>0:
            price.update(safe)
        if len(imgg)>0:
            price.update(imgg)
        if len(dsheet)>0:
            price.update(dsheet)
        if len(kitty)>0:
            price.update(kitty)
            
        empty.append(price)
    print("process no "+str(num)+"completed")
    time.sleep(1)
        
        

process no 0completed
process no 1completed
process no 2completed
process no 3completed
process no 4completed
process no 5completed
process no 6completed
process no 7completed
process no 8completed
process no 9completed
process no 10completed
process no 11completed
process no 12completed
process no 13completed


In [275]:
new=[]
for i in empty:
    new.append(i)

In [276]:
buffer=pd.DataFrame(new)
buffer.columns

Index([                   'Price',            'Product Title',
                            'Sku',              'Product Url',
                     'Categories',                  'Isotype',
                  'Product Type ',                    'Units',
                           'Host',             'Application ',
                             None,               'Background',
                   'Applications',                  'Storage',
                        'Caution',               'References',
                     'Kit Manual',       'Safety Datasheet_1',
             'Safety Datasheet_2',                'Datasheet',
                'Kit Manual_link',                  'Product',
                ['Product Form:'],   ['Purification Method:'],
             ['Incubation Time:'], ['Working Concentration:'],
                               ''],
      dtype='object')

In [277]:
buffer.columns=[                   'Price',            'Product Title',
                            'Sku',              'Product Url',
                     'Categories',                  'Isotype',
                  'Product Type ',                    'Units',
                           'Host',             'Application ',
                             "None",               'Background',
                   'Applications',                  'Storage',
                        'Caution',               'References',
                     'Kit Manual',       'Safety Datasheet_1',
             'Safety Datasheet_2',                'Datasheet',
                'Kit Manual_link',                  'Product',
                'Product Form',   'Purification Method',
             'Incubation Time', 'Working Concentration',
                               '']

In [278]:
buffer

,Price,Product Title,Sku,Product Url,Categories,Isotype,Product Type,Units,Host,Application,...,Safety Datasheet_1,Safety Datasheet_2,Datasheet,Kit Manual_link,Product,Product Form,Purification Method,Incubation Time,Working Concentration,
0,"""242.00""",BrdU Cell Proliferation Assay Kit,X1327K1,https://www.exalpha.com/product/brdu-cell-prol...,"Categories: Assay Kit, BrdU assay kits, Kits",N/A,Assay Kit,200 Tests,N/A,ELISA,...,/wp-content/uploads/2018/06/X1327K-BrdU-ELISA-...,/wp-content/uploads/SDS/X1327K SDS_V3.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
1,"""535.00""",BrdU Cell Proliferation Assay Kit,X1327K2,https://www.exalpha.com/product/brdu-cell-prol...,"Categories: Assay Kit, BrdU assay kits, Kits",N/A,Assay Kit,1000 Tests,N/A,ELISA,...,/wp-content/uploads/2018/06/X1327K-BrdU-ELISA-...,/wp-content/uploads/SDS/X1327K SDS_V3.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
2,"""1929.00""",BrdU Cell Proliferation Assay Kit,X1327K3,https://www.exalpha.com/product/brdu-cell-prol...,"Categories: Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,5000 Tests,NaN,ELISA,...,/wp-content/uploads/2018/06/X1327K-BrdU-ELISA-...,/wp-content/uploads/SDS/X1327K SDS_V3.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
3,"""297.00""",BrdU Chemiluminescent Cell Proliferation Assay...,X1623K1,https://www.exalpha.com/product/brdu-chemilumi...,"Categories: Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,200 Tests,NaN,ELISA,...,/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-...,/wp-content/uploads/SDS/X1623K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
4,"""581.00""",BrdU Chemiluminescent Cell Proliferation Assay...,X1623K2,https://www.exalpha.com/product/brdu-chemilumi...,"Categories: Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,1000 Tests,NaN,NaN,...,/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-...,/wp-content/uploads/SDS/X1623K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
5,"""2101.00""",BrdU Chemiluminescent Cell Proliferation Assay...,X1623K3,https://www.exalpha.com/product/brdu-chemilumi...,"Categories: Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,5000 Tests,NaN,ELISA,...,/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-...,/wp-content/uploads/SDS/X1623K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
6,"""399.00""",BrdU Immunohistochemistry Kit,X1545K,https://www.exalpha.com/product/brdu-immunohis...,"Categories: BrdU assay kits, Immunohistochemis...",NaN,Immunohistochemistry Kit,50 Slides,NaN,Immunohistochemistry on paraffin embedded and ...,...,/wp-content/uploads/2018/05/X1545K-PDS-v1.0.pdf,/wp-content/uploads/SDS/X1545K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
7,"""215.00""",DNA Fragmentation Detection Kit,X2044K1,https://www.exalpha.com/product/dna-fragmentat...,"Categories: Apoptosis kits, Immunohistochemist...",NaN,Immunohistochemistry Kit,30 Tests,NaN,Immunohistochemistry,...,/wp-content/uploads/2018/05/X2044K-PDS-v1.0.pdf,/wp-content/uploads/SDS/X2044K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
8,"""413.00""",DNA Fragmentation Detection Kit,X2044K2,https://www.exalpha.com/product/dna-fragmentat...,"Categories: Apoptosis kits, Immunohistochemist...",NaN,Immunohistochemistry Kit,60 Tests,NaN,Immunohistochemistry,...,/wp-content/uploads/2018/05/X2044K-PDS-v1.0.pdf,/wp-content/uploads/SDS/X2044K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
9,"""679.00""",EggsPress IgY Purification Kit

In [279]:
for i in buffer.columns:
    buffer[i]=buffer[i].str.replace(i,"")
    buffer[i]=buffer[i].str.replace('"',"")
    buffer[i]=buffer[i].str.replace('\n',"")
    #buffer[i]=buffer[i].str.replace(':',"")
buffer    

,Price,Product Title,Sku,Product Url,Categories,Isotype,Product Type,Units,Host,Application,...,Safety Datasheet_1,Safety Datasheet_2,Datasheet,Kit Manual_link,Product,Product Form,Purification Method,Incubation Time,Working Concentration,
0,242.00,BrdU Cell Proliferation Assay Kit,X1327K1,https://www.exalpha.com/product/brdu-cell-prol...,": Assay Kit, BrdU assay kits, Kits",N/A,Assay Kit,200 Tests,N/A,ELISA,...,/wp-content/uploads/2018/06/X1327K-BrdU-ELISA-...,/wp-content/uploads/SDS/X1327K SDS_V3.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
1,535.00,BrdU Cell Proliferation Assay Kit,X1327K2,https://www.exalpha.com/product/brdu-cell-prol...,": Assay Kit, BrdU assay kits, Kits",N/A,Assay Kit,1000 Tests,N/A,ELISA,...,/wp-content/uploads/2018/06/X1327K-BrdU-ELISA-...,/wp-content/uploads/SDS/X1327K SDS_V3.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
2,1929.00,BrdU Cell Proliferation Assay Kit,X1327K3,https://www.exalpha.com/product/brdu-cell-prol...,": Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,5000 Tests,NaN,ELISA,...,/wp-content/uploads/2018/06/X1327K-BrdU-ELISA-...,/wp-content/uploads/SDS/X1327K SDS_V3.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
3,297.00,BrdU Chemiluminescent Cell Proliferation Assay...,X1623K1,https://www.exalpha.com/product/brdu-chemilumi...,": Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,200 Tests,NaN,ELISA,...,/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-...,/wp-content/uploads/SDS/X1623K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
4,581.00,BrdU Chemiluminescent Cell Proliferation Assay...,X1623K2,https://www.exalpha.com/product/brdu-chemilumi...,": Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,1000 Tests,NaN,NaN,...,/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-...,/wp-content/uploads/SDS/X1623K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
5,2101.00,BrdU Chemiluminescent Cell Proliferation Assay...,X1623K3,https://www.exalpha.com/product/brdu-chemilumi...,": Assay Kit, BrdU assay kits, Kits",NaN,Assay Kit,5000 Tests,NaN,ELISA,...,/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-...,/wp-content/uploads/SDS/X1623K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
6,399.00,BrdU Immunohistochemistry Kit,X1545K,https://www.exalpha.com/product/brdu-immunohis...,": BrdU assay kits, Immunohistochemistry Kit, Kits",NaN,Immunohistochemistry Kit,50 Slides,NaN,Immunohistochemistry on paraffin embedded and ...,...,/wp-content/uploads/2018/05/X1545K-PDS-v1.0.pdf,/wp-content/uploads/SDS/X1545K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
7,215.00,DNA Fragmentation Detection Kit,X2044K1,https://www.exalpha.com/product/dna-fragmentat...,": Apoptosis kits, Immunohistochemistry Kit, Kits",NaN,Immunohistochemistry Kit,30 Tests,NaN,Immunohistochemistry,...,/wp-content/uploads/2018/05/X2044K-PDS-v1.0.pdf,/wp-content/uploads/SDS/X2044K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
8,413.00,DNA Fragmentation Detection Kit,X2044K2,https://www.exalpha.com/product/dna-fragmentat...,": Apoptosis kits, Immunohistochemistry Kit, Kits",NaN,Immunohistochemistry Kit,60 Tests,NaN,Immunohistochemistry,...,/wp-content/uploads/2018/05/X2044K-PDS-v1.0.pdf,/wp-content/uploads/SDS/X2044K SDS_V1.pdf,/wp-content/uploads/2018/04/pds.png,https://www.exalpha.com/wp-content/uploads/201...,NaN,NaN,NaN,NaN,NaN,NaN
9,679.00,EggsPress IgY Purification Kit (20 yolks),IK 2000,https://www.exalpha.com/product/eggspress-igy-...,": Antibody Purification Kit, Kits, Kits",NaN,Kits,N

In [280]:
u=[]
buffer["Safety Datasheet_1"]=buffer["Safety Datasheet_1"].str.replace(" ","%20")
for n,i in enumerate(buffer["Safety Datasheet_1"]):
    if "/" in str(i):
        k="https://www.exalpha.com"+str(i)
        u.append(k)
    else:
        u.append(str(i))
buffer["Safety Datasheet_1"]=u        

In [281]:
u=[]
buffer["Safety Datasheet_2"]=buffer["Safety Datasheet_2"].str.replace(" ","%20")
for n,i in enumerate(buffer["Safety Datasheet_2"]):
    if "/" in str(i):
        k="https://www.exalpha.com"+str(i)
        u.append(k)
    else:
        u.append(str(i))
buffer["Safety Datasheet_2"]=u
buffer["Safety Datasheet_2"]

0     https://www.exalpha.com/wp-content/uploads/SDS...
1     https://www.exalpha.com/wp-content/uploads/SDS...
2     https://www.exalpha.com/wp-content/uploads/SDS...
3     https://www.exalpha.com/wp-content/uploads/SDS...
4     https://www.exalpha.com/wp-content/uploads/SDS...
5     https://www.exalpha.com/wp-content/uploads/SDS...
6     https://www.exalpha.com/wp-content/uploads/SDS...
7     https://www.exalpha.com/wp-content/uploads/SDS...
8     https://www.exalpha.com/wp-content/uploads/SDS...
9                                                   nan
10                                                  nan
11                                                  nan
12                                                  nan
13                                                  nan
Name: Safety Datasheet_2, dtype: object

In [282]:

u=[]
buffer["Datasheet"]=buffer["Datasheet"].str.replace(" ","%20")
for n,i in enumerate(buffer["Datasheet"]):
    if "/" in str(i):
        k="https://www.exalpha.com"+str(i)
        u.append(k)
    else:
        u.append(str(i))
buffer["Datasheet"]=u
buffer["Datasheet"]

0     https://www.exalpha.com/wp-content/uploads/201...
1     https://www.exalpha.com/wp-content/uploads/201...
2     https://www.exalpha.com/wp-content/uploads/201...
3     https://www.exalpha.com/wp-content/uploads/201...
4     https://www.exalpha.com/wp-content/uploads/201...
5     https://www.exalpha.com/wp-content/uploads/201...
6     https://www.exalpha.com/wp-content/uploads/201...
7     https://www.exalpha.com/wp-content/uploads/201...
8     https://www.exalpha.com/wp-content/uploads/201...
9     https://www.exalpha.com/wp-content/uploads/201...
10    https://www.exalpha.com/wp-content/uploads/201...
11    https://www.exalpha.com/wp-content/uploads/201...
12    https://www.exalpha.com/wp-content/uploads/201...
13    https://www.exalpha.com/wp-content/uploads/201...
Name: Datasheet, dtype: object

In [283]:
buffer.drop(["Kit Manual","Safety Datasheet_1"],inplace=True,axis=1)

In [284]:
#print(buffer["Safety Datasheet_1"][3])
print(buffer["Kit Manual_link"][3])

https://www.exalpha.com/wp-content/uploads/2018/07/X1623K-BrdU-Chemi-ELISA-Manual_V2.1.pdf


In [285]:
buffer.to_excel("kits(exalpha).xlsx",header=True,index=False)

# for primary antibodies only

In [195]:
df=pd.read_excel("ampleanti(exalpha).xlsx")
df.columns

Index(['Price', 'Product Title', 'Sku', 'Product Url', 'Categories', 'Units',
       'Host', 'Application ', 'Unnamed: 8', 'Background', 'Source',
       '<i>Immunogen:</i>', 'Product', '<i>Purification Method:</i>',
       '<i>Concentration:</i>', 'Specificity', '<i>Species Reactivity:</i>',
       'Applications', '<i>Working Concentration:</i>',
       '<i>Positive Control:</i>', 'Storage', 'Caution', 'References',
       'Safety Datasheet(s) for this product:', 'Safety Datasheet_1',
       'Datasheet', 'Product Type ', 'Species reactivity ',
       '<i>Secondary Reagents:</i>', '<i>Incubation Time:</i>', 'Clone',
       'Isotype', '<i>Pre-Treatment:</i>', 'Protein Reference(s)',
       '<i>Database Name:</i>', '<i>Accession number:</i>',
       '<i>Species Accession:</i>', '<i>Formulation:</i>', 'Unnamed: 38',
       '<i>Product Form:</i>', 'Safety Datasheet_2', 'Image_1',
       'Image_1_description', '<i>Shipping Conditions:</i>', 'Warranty',
       'CE Mark', 'Image_2', 'Image_2_

In [197]:
col=[]
for i in df.columns:
    if '<i>' in i:
        i=i.replace('<i>','')
        i=i.replace('</i>','')
        col.append(i)
    else:
        col.append(i)
df.columns=col        

In [199]:
df

,Price,Product Title,Sku,Product Url,Categories,Units,Host,Application,Unnamed: 8,Background,...,Image_7,Image_7_description,Image_8,Image_8_description,Image_9,Image_9_description,Image_10,Image_10_description,Image_11,Image_11_description
0,496.0,25(OH)Vitamin D3-1-alpha-Hydroxylase,HYD001,https://www.exalpha.com/product/25ohvitamin-d3...,": Primary antibodies, Primary antibodies",100 µl,Sheep,Immunohistochemistry Western Blotting,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundThe polyclonal antibody is well su...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1156.0,"AGE, Advanced Glycation End Products",AGE102-1.0,https://www.exalpha.com/product/age-advanced-g...,": Primary antibodies, Primary antibodies",1 ml,Rabbit,ELISA IHC not established,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundThis antibody is suitable for the ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,249.0,"Alpha-Actin, sarcomeric",AA505,https://www.exalpha.com/product/alpha-actin-sa...,": Primary antibodies, Primary antibodies",5 ml,Mouse,IHC(C Immunohistochemistry (frozen) Immunohi...,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundThe antibody 5C5 which is directed...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,926.0,"Alpha-DEFENSIN (1-3), HNP1-3",HNP103,https://www.exalpha.com/product/alpha-defensin...,": Primary antibodies, Primary antibodies",100 µg,Mouse,Flow Cytometry IHC(C Immunohistochemistry (f...,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundThe monoclonal antibody DEF-3 rea...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,595.0,AMACR (P504S) clone 13H4,P504S-0.5,https://www.exalpha.com/product/amacr-p504s-cl...,": Primary antibodies, Primary antibodies",0.5 ml,Rabbit,IHC(C Immunohistochemistry (frozen) Immunohi...,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundThe monoclonal Rabbit antibody dir...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,193.0,"Swine anti Human Serum proteins, conjugated wi...",SwAHu/TSP/FITC,https://www.exalpha.com/product/swine-anti-hum...,": Primary antibodies, Primary antibodies",1 ml,Swine,NaN,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundIn immunoelectrophoresis against n...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,546.0,Tenascin C (TNC),TEN001,https://www.exalpha.com/product/tenascin-c-tnc/,": Primary antibodies, Primary antibodies",1 ml,Mouse,IHC(C Immunohistochemistry (frozen) Immunohi...,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundThe antibody (T2H5) reacts with at...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,331.0,Tiam 1 (Catalytic domain/ C-terminal domain),MUB1807P,https://www.exalpha.com/product/tiam-1-catalyt...,": Primary antibodies, Primary antibodies",0.1ml,Rabbit,Immunoblotting Immunocytochemistry Immunohis...,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundDirectional cell migration is esse...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,222.0,Tiam 1 (N-terminal domain),MUB1808S,https://www.exalpha.com/product/tiam-1-n-termi...,": Primary antibodies, Primary antibodies",0.25ml,Rabbit,Immunoblotting Immunocytochemistry Immunohis...,2017 © Exalpha Biologicals / Sales and Deliv...,\nBackgroundDirectional cell migration is esse...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [201]:
df.to_excel("primary_antibodies(exalpha).xlsx",header=True)